In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [14]:
from __future__ import absolute_import, division, print_function
tf.enable_eager_execution()

In [3]:
tf.executing_eagerly()
x=[[2.]]
m=tf.matmul(x,x)
print(m)

tf.Tensor([[ 4.]], shape=(1, 1), dtype=float32)


In [8]:
m.shape(0)

TensorShape([Dimension(1), Dimension(1)])

In [10]:
t = tf.constant([[1, 4, 1],
                 [3, 5, 3],
                 [5, 6, 5]])

In [11]:
s = tf.slice(t, [0, 1], [-1,2])
print("s:%s%s"%(s.shape,s.shape))
s.shape

s:(3, 2)(3, 2)


TensorShape([Dimension(3), Dimension(2)])

In [28]:
t.get_shape().as_list()

[3, 3]

In [38]:
user = np.random.random((2, 5))
video = np.random.random((10, 5))
norm_user = tf.tile(tf.sqrt(tf.reduce_sum(tf.square(user), 1, True)), [5, 1])
norm_video = tf.sqrt(tf.reduce_sum(tf.square(video), 1, True))

prod = tf.reduce_sum(tf.multiply(tf.tile(user, [5, 1]), video), 1, True)
norm_prod = tf.multiply(norm_user, norm_video)
cos_sim_raw = tf.truediv(prod, norm_prod)
cos_sim = tf.transpose(tf.reshape(tf.transpose(cos_sim_raw),
                                  [5, 2]))

print(cos_sim)
prob = tf.nn.softmax(cos_sim)
print(prob)
hit_prob = tf.slice(prob, [0, 0], [-1, 1])
loss = -tf.reduce_sum(tf.log(tf.clip_by_value(hit_prob, 1e-8, 1))) / 2
loss

tf.Tensor(
[[ 0.81759865  0.5047507   0.59747467  0.68677071  0.58088591]
 [ 0.98713083  0.91984914  0.92832989  0.82119963  0.78849004]], shape=(2, 5), dtype=float64)
tf.Tensor(
[[ 0.23807443  0.17411838  0.19103552  0.20887906  0.18789262]
 [ 0.22003289  0.20571575  0.20746779  0.18639085  0.18039272]], shape=(2, 5), dtype=float64)


<tf.Tensor: id=887, shape=(), dtype=float64, numpy=1.4745750851710648>

<tf.Tensor: id=106, shape=(), dtype=float64, numpy=2.8354963844330725>

In [21]:
tf.reshape(t,[-1, 3])

<tf.Tensor: id=41, shape=(3, 3), dtype=int32, numpy=
array([[1, 4, 1],
       [3, 5, 3],
       [5, 6, 5]])>

In [68]:
x=[1,2,3,4,5,6,7,8,9]
m=tf.multiply(x, 1)
mr=tf.reshape(m, [3,3])
print(mr)
mrr=tf.reshape(mr, [-1, 3])
mrr

tf.Tensor(
[[1 2 3]
 [4 5 6]
 [7 8 9]], shape=(3, 3), dtype=int32)


<tf.Tensor: id=268, shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])>

In [84]:
x=[[[1, 1, 1], [2, 2, 2]],[[3, 3, 3], [4, 4, 4]],[[5, 5, 5], [6, 6, 6]]]
m=tf.multiply(x, 1)
print(m)
mm=tf.slice(m, [0,0,0],[2,1,3])
print(mm)
tf.expand_dims(mm, -1)


tf.Tensor(
[[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]

 [[5 5 5]
  [6 6 6]]], shape=(3, 2, 3), dtype=int32)
tf.Tensor(
[[[1 1 1]]

 [[3 3 3]]], shape=(2, 1, 3), dtype=int32)


<tf.Tensor: id=382, shape=(2, 1, 3, 1), dtype=int32, numpy=
array([[[[1],
         [1],
         [1]]],


       [[[3],
         [3],
         [3]]]])>

In [11]:
datas = pd.read_csv("dataset/train06.csv", header=None)
datas.head()

,0,1,2,3,4,5,6
0,15.341,0.079,0.000,229,NaN,1497974446,0
1,0.000,0.067,0.084,5,305.0,1498262726,0
2,7.001,0.074,0.094,39,583.0,1498184175,0
3,3.000,0.097,0.125,75,574.0,1498117360,0
4,19.162,0.181,0.120,59,NaN,1498199138,0


In [29]:
features=datas.iloc[:, 0:6].values
labels=datas.iloc[:,6].values

In [33]:
np.savez('dataset/data.npy', features=datas.iloc[:, 0:6].values, labels=datas.iloc[:,6].values)

In [20]:
dic = {'features':datas.iloc[:,0:6], 'labels':datas[6]}
with open('dataset/data.pkl', 'wb') as f:
    pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

In [34]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("dataset/data.npy.npz") as data:
    features = data["features"]
    labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [35]:
dataset

<TensorSliceDataset shapes: ((6,), ()), types: (tf.float64, tf.int64)>

In [4]:
w = tf.contrib.eager.Variable([[2.0]])
with tf.GradientTape() as tape:
    loss = w*w

grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[4.]], shape=(1, 1), dtype=float32)


In [37]:
dataset = tf.data.TFRecordDataset('dataset/train06.csv')
dataset = dataset.batch(30)

In [39]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
#dataset = dataset.map(...)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.

# Initialize `iterator` with training data.
training_filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
sess.run(iterator.initializer, feed_dict={filenames: training_filenames})

# Initialize `iterator` with validation data.
validation_filenames = ["/var/data/validation1.tfrecord", ...]
sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [19]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
  return input * weight + bias

# A loss function using mean-squared error
def loss(weights, biases):
  error = prediction(training_inputs, weights, biases) - training_outputs
  return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
  with tf.GradientTape() as tape:
    loss_value = loss(weights, biases)
  return tape.gradient(loss_value, [weights, biases])

train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tf.contrib.eager.Variable(5.)
B = tf.contrib.eager.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
  dW, dB = grad(W, B)
  W.assign_sub(dW * learning_rate)
  B.assign_sub(dB * learning_rate)
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

Initial loss: 68.393
Loss at step 000: 65.753
Loss at step 020: 30.109
Loss at step 040: 14.079
Loss at step 060: 6.867
Loss at step 080: 3.621
Loss at step 100: 2.160
Loss at step 120: 1.502
Loss at step 140: 1.206
Loss at step 160: 1.072
Loss at step 180: 1.012
Final loss: 0.986
W = 3.0677196979522705, B = 2.126572847366333


In [7]:
import dataset
dataset_train = dataset.train('../MNIST/dataset/')

ok


In [8]:
dataset_train

<ZipDataset shapes: ((784,), ()), types: (tf.float32, tf.int32)>

In [9]:
def loss(model, x, y):
  prediction = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=prediction)

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

x, y = iter(dataset_train).next()
print("Initial loss: {:.3f}".format(loss(model, x, y)))

# Training loop
for (i, (x, y)) in enumerate(dataset_train):
  # Calculate derivatives of the input function with respect to its parameters.
  grads = grad(model, x, y)
  # Apply the gradient to the model
  optimizer.apply_gradients(zip(grads, model.variables),
                            global_step=tf.train.get_or_create_global_step())
  if i % 200 == 0:
    print("Loss at step {:04d}: {:.3f}".format(i, loss(model, x, y)))

print("Final loss: {:.3f}".format(loss(model, x, y)))

TypeError: __init__() missing 2 required positional arguments: 'message' and 'code'

In [4]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))

In [5]:
dataset1

<TensorSliceDataset shapes: (10,), types: tf.float32>

In [6]:
print(dataset1)

<TensorSliceDataset shapes: (10,), types: tf.float32>


In [7]:
tf.random_uniform([4, 10])

<tf.Tensor: id=16, shape=(4, 10), dtype=float32, numpy=
array([[0.3068024 , 0.42981327, 0.00752783, 0.28380334, 0.21418095,
        0.5036495 , 0.7795782 , 0.63594496, 0.3038565 , 0.7676972 ],
       [0.33896303, 0.9078715 , 0.0778718 , 0.46068144, 0.0718416 ,
        0.99612415, 0.39350307, 0.39160836, 0.942485  , 0.9438869 ],
       [0.31136394, 0.882895  , 0.14068246, 0.5082891 , 0.10826325,
        0.28466737, 0.4340589 , 0.18669868, 0.7672312 , 0.5167384 ],
       [0.21945596, 0.11692297, 0.85190547, 0.08116734, 0.6356126 ,
        0.3887303 , 0.95782626, 0.37064326, 0.34053433, 0.9378414 ]],
      dtype=float32)>

In [12]:
with tf.Session() as sess:
    dataset = tf.data.Dataset.range(100)
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    for i in range(100):
        value = sess.run(next_element)
        assert i == value

In [45]:
with tf.Session() as sess:
    dataset = tf.data.Dataset.range(100)
    dataset = dataset.shuffle(1).repeat().batch(10)
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    print(sess.run(next_element))

    for i in range(200):
        value = sess.run(next_element)
        print(value)
        #assert i == value

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89]
[90 91 92 93 94 95 96 97 98 99]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89]
[90 91 92 93 94 95 96 97 98 99]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[50 51 52 53 54 55 56 57 58 59]
[60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89]
[90 91 92 93 94 95 96 97 98 99]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 

In [17]:
with tf.Session() as sess:
    # Define training and validation datasets with the same structure.
    training_dataset = tf.data.Dataset.range(100).map(
        lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
    validation_dataset = tf.data.Dataset.range(50)
    print(training_dataset)
    print(validation_dataset)

    # A reinitializable iterator is defined by its structure. We could use the
    # `output_types` and `output_shapes` properties of either `training_dataset`
    # or `validation_dataset` here, because they are compatible.
    iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                               training_dataset.output_shapes)
    next_element = iterator.get_next()

    training_init_op = iterator.make_initializer(training_dataset)
    validation_init_op = iterator.make_initializer(validation_dataset)

    # Run 20 epochs in which the training dataset is traversed, followed by the
    # validation dataset.
    for _ in range(20):
        # Initialize an iterator over the training dataset.
        sess.run(training_init_op)
        for _ in range(100):
            sess.run(next_element)
            print(next_element)

        # Initialize an iterator over the validation dataset.
        sess.run(validation_init_op)
        for _ in range(50):
            sess.run(next_element)

<MapDataset shapes: (), types: tf.int64>
<RangeDataset shapes: (), types: tf.int64>
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("Iterato

Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=

Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=

Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=

Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=

Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=int64)
Tensor("IteratorGetNext_6:0", shape=(), dtype=

In [ ]:
with tf.Session() as sess:
    # Define training and validation datasets with the same structure.
    training_dataset = tf.data.Dataset.range(100).map(
        lambda x: x + tf.random_uniform([], -10, 10, tf.int64)).repeat()
    validation_dataset = tf.data.Dataset.range(50)

    # A feedable iterator is defined by a handle placeholder and its structure. We
    # could use the `output_types` and `output_shapes` properties of either
    # `training_dataset` or `validation_dataset` here, because they have
    # identical structure.
    handle = tf.placeholder(tf.string, shape=[])
    iterator = tf.data.Iterator.from_string_handle(
        handle, training_dataset.output_types, training_dataset.output_shapes)
    next_element = iterator.get_next()

    # You can use feedable iterators with a variety of different kinds of iterator
    # (such as one-shot and initializable iterators).
    training_iterator = training_dataset.make_one_shot_iterator()
    validation_iterator = validation_dataset.make_initializable_iterator()

    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())

    # Loop forever, alternating between training and validation.
    while True:
      # Run 200 steps using the training dataset. Note that the training dataset is
      # infinite, and we resume from where we left off in the previous `while` loop
      # iteration.
      for _ in range(200):
        sess.run(next_element, feed_dict={handle: training_handle})

      # Run one pass over the validation dataset.
        sess.run(validation_iterator.initializer)
        for _ in range(50):
            sess.run(next_element, feed_dict={handle: validation_handle})